# INFO 2950 FINAL PROJECT: Predicting Flight Delays and Cancellations in the US

# __RESEARCH QUESTION:__
What factors affect if a U.S domestic flight will be delayed or cancelled? Which flights are most likely to be delayed or cancelled? Are there airlines that are better than others in terms of number of delays and delay times? Can flight history be used to predict the delay time of future flights?

In this assignment we will see if we can use our data from 2018, about if a flight has been delayed or not and the various factors affecting it. We will use values such as various values such as Cancelled, Diverted, Month, Operating Airline, Distance and originState and many more to see if we can predict if a flight will be delayed or cancelled, and possibly by how much.

## *Table of Contents*
1. Research Question
2. Data Cleaning
3. Exploratory Analysis
4. Data description <br>
    a. Motivation <br>
    b. Composition<br>
    c. Collection Process <br>
5. Phase 4 <br>
    a. Hypothesis 1 <br>
    b. Hypothesis 2 <br>
6. Phase 5
    a. Hypothesis 2 <br>
7. Data Limitation
8. Interpretation and Conclusions
9. Future Research
10. Refrences
11. Appendix

## Data Cleaning:

We first imported our flights_2022 dataset. As can be seen on out GIT. The filed were very large thus have been split into part 1 and part2. This will make the analysis much easier. 
We first loaded the dataset as flight_2018.
As can be seen the head of both of these datsets have been printed. 

In [46]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve

The data we are importing for our project is data from 2018. Because the file was very large to store in Github. We randomly smapled 10% of the original data. This data is stored in Flights_2018_modified. The code for this procedure can be found in the Appendix.

In [47]:

flights_2018=pd.read_csv('Flights_2018_Modified (1).csv')
airports = pd.read_csv('airports.csv')

In [48]:
flights_2018.head()

,Unnamed: 0,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,32314,2018-01-01,Hawaiian Airlines Inc.,KOA,HNL,False,False,1543,1546.0,3.0,...,1555.0,1626.0,8.0,1632,2.0,0.0,0.0,1600-1659,1,0.0
1,223193,2018-01-01,Delta Air Lines Inc.,IND,ATL,False,False,937,943.0,6.0,...,1007.0,1113.0,4.0,1119,-2.0,0.0,-1.0,1100-1159,2,0.0
2,349735,2018-01-01,Allegiant Air,IAG,PGD,False,False,1043,1043.0,0.0,...,1111.0,1350.0,4.0,1341,13.0,0.0,0.0,1300-1359,5,0.0
3,263507,2018-01-01,Southwest Airlines Co.,LAS,OMA,False,False,2125,2200.0,35.0,...,2211.0,233.0,8.0,155,46.0,1.0,3.0,0001-0559,5,0.0
4,223533,2018-01-01,Delta Air Lines Inc.,ATL,BOS,False,False,1248,1247.0,0.0,...,1303.0,1454.0,16.0,1513,-3.0,0.0,-1.0,1500-1559,4,0.0


In [49]:
flights_2018.head()

,Unnamed: 0,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,32314,2018-01-01,Hawaiian Airlines Inc.,KOA,HNL,False,False,1543,1546.0,3.0,...,1555.0,1626.0,8.0,1632,2.0,0.0,0.0,1600-1659,1,0.0
1,223193,2018-01-01,Delta Air Lines Inc.,IND,ATL,False,False,937,943.0,6.0,...,1007.0,1113.0,4.0,1119,-2.0,0.0,-1.0,1100-1159,2,0.0
2,349735,2018-01-01,Allegiant Air,IAG,PGD,False,False,1043,1043.0,0.0,...,1111.0,1350.0,4.0,1341,13.0,0.0,0.0,1300-1359,5,0.0
3,263507,2018-01-01,Southwest Airlines Co.,LAS,OMA,False,False,2125,2200.0,35.0,...,2211.0,233.0,8.0,155,46.0,1.0,3.0,0001-0559,5,0.0
4,223533,2018-01-01,Delta Air Lines Inc.,ATL,BOS,False,False,1248,1247.0,0.0,...,1303.0,1454.0,16.0,1513,-3.0,0.0,-1.0,1500-1559,4,0.0


We will be using 1000 random rows for now because our dataset is too large. We will figure out how to use all of the data for future parts of the project.

We have also loaded the airports dataset aairports. This datset contains the IATA, Airport, City, State, Country and Latitude and Longitude. As most of this dataset was very clean, we did not do much cleaning. 
The only cleaning we did was remove the colunm Country as all this data is for US, and is redundent. 
The code below is the code before Country is dropped. 

In [50]:
print(airports.shape,'\n',airports.columns)

(341, 7) 
 Index(['IATA', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE'], dtype='object')


In [51]:
airports.head()

,IATA,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
1,ANC,Ted Stevens Anchorage International,Anchorage,AK,USA,61.174320,-149.996186
2,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944
3,AUS,Austin-Bergstrom International,Austin,TX,USA,30.194533,-97.669872
4,BDL,Bradley International,Windsor Locks,CT,USA,41.938874,-72.683228


The code below removes the colunm COUNTRY.

In [52]:
airports = airports.drop('COUNTRY', axis=1)
airports.head()

,IATA,AIRPORT,CITY,STATE,LATITUDE,LONGITUDE
0,ABQ,Albuquerque International,Albuquerque,NM,35.040222,-106.609194
1,ANC,Ted Stevens Anchorage International,Anchorage,AK,61.174320,-149.996186
2,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944
3,AUS,Austin-Bergstrom International,Austin,TX,30.194533,-97.669872
4,BDL,Bradley International,Windsor Locks,CT,41.938874,-72.683228


In the cells below we have used SQL to left join, the origin airports in the flights_2022 dataset to the IATA airports in the airports data. We have used duckDB and selected all the colunms as well as rebamed some colunms such as: airport.city as origin_city and airports_latitude as origin latitude to make it easier.

In [53]:
df = duckdb.sql('''
SELECT flights_2018.*
, airports.city AS origin_city
, airports.state AS origin_state
, airports.latitude AS origin_latitude
, airports.longitude AS origin_longitude
FROM flights_2018
LEFT JOIN airports ON flights_2018.origin = airports.IATA
''').df()

In [54]:
df = duckdb.sql('''
SELECT flights_2018.*
, airports.city AS origin_city
, airports.state AS origin_state
, airports.latitude AS origin_latitude
, airports.longitude AS origin_longitude
FROM flights_2018
LEFT JOIN airports ON flights_2018.origin = airports.IATA
''').df()

In [55]:
df2 = duckdb.sql('''
SELECT df.*
, airports.city AS dest_city
, airports.state AS dest_state
, airports.latitude AS dest_latitude
, airports.longitude AS dest_longitude
FROM df
LEFT JOIN airports ON df.dest = airports.IATA
''').df()

This is the output of the head (first 5 rows of the final dataframe df2)

In [56]:
df2.head()

,Unnamed: 0,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,...,DistanceGroup,DivAirportLandings,origin_city,origin_state,origin_latitude,origin_longitude,dest_city,dest_state,dest_latitude,dest_longitude
0,4327301,2018-07-25,Southwest Airlines Co.,BUF,MCO,False,False,1850,1849.0,0.0,...,5,0.0,Buffalo,NY,42.940525,-78.732167,Orlando,FL,28.428889,-81.316028
1,4327335,2018-07-25,Southwest Airlines Co.,BUR,PHX,False,False,1555,1624.0,29.0,...,2,0.0,Burbank,CA,34.200619,-118.358497,Phoenix,AZ,33.434167,-112.008056
2,4510605,2018-07-25,Southwest Airlines Co.,SAT,PHX,False,False,1120,1131.0,11.0,...,4,0.0,San Antonio,TX,29.533694,-98.469778,Phoenix,AZ,33.434167,-112.008056
3,4535524,2018-07-25,Horizon Air,SFO,SNA,False,False,926,929.0,3.0,...,2,0.0,San Francisco,CA,37.619002,-122.374843,Santa Ana,CA,33.675659,-117.868223
4,4327812,2018-07-25,Southwest Airlines Co.,DAL,PHL,False,False,1225,1323.0,58.0,...,6,0.0,Dallas,TX,32.847114,-96.851772,Philadelphia,PA,39.871953,-75.241141


In [57]:
df2.head()

,Unnamed: 0,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,...,DistanceGroup,DivAirportLandings,origin_city,origin_state,origin_latitude,origin_longitude,dest_city,dest_state,dest_latitude,dest_longitude
0,4327301,2018-07-25,Southwest Airlines Co.,BUF,MCO,False,False,1850,1849.0,0.0,...,5,0.0,Buffalo,NY,42.940525,-78.732167,Orlando,FL,28.428889,-81.316028
1,4327335,2018-07-25,Southwest Airlines Co.,BUR,PHX,False,False,1555,1624.0,29.0,...,2,0.0,Burbank,CA,34.200619,-118.358497,Phoenix,AZ,33.434167,-112.008056
2,4510605,2018-07-25,Southwest Airlines Co.,SAT,PHX,False,False,1120,1131.0,11.0,...,4,0.0,San Antonio,TX,29.533694,-98.469778,Phoenix,AZ,33.434167,-112.008056
3,4535524,2018-07-25,Horizon Air,SFO,SNA,False,False,926,929.0,3.0,...,2,0.0,San Francisco,CA,37.619002,-122.374843,Santa Ana,CA,33.675659,-117.868223
4,4327812,2018-07-25,Southwest Airlines Co.,DAL,PHL,False,False,1225,1323.0,58.0,...,6,0.0,Dallas,TX,32.847114,-96.851772,Philadelphia,PA,39.871953,-75.241141


In [58]:
#Selecting only specified colunms from df2. 
cols = ['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'DivAirportLandings', 'origin_latitude', 'origin_longitude', 'dest_latitude', 'dest_longitude']
new_df = df2[cols]

In [59]:
new_df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,origin_latitude,origin_longitude,dest_latitude,dest_longitude
0,2018-07-25,Southwest Airlines Co.,BUF,MCO,False,False,1850,1849.0,0.0,-1.0,...,0.0,0.0,0.0,2100-2159,5,0.0,42.940525,-78.732167,28.428889,-81.316028
1,2018-07-25,Southwest Airlines Co.,BUR,PHX,False,False,1555,1624.0,29.0,29.0,...,42.0,1.0,2.0,1700-1759,2,0.0,34.200619,-118.358497,33.434167,-112.008056
2,2018-07-25,Southwest Airlines Co.,SAT,PHX,False,False,1120,1131.0,11.0,11.0,...,3.0,0.0,0.0,1100-1159,4,0.0,29.533694,-98.469778,33.434167,-112.008056
3,2018-07-25,Horizon Air,SFO,SNA,False,False,926,929.0,3.0,3.0,...,-17.0,0.0,-2.0,1100-1159,2,0.0,37.619002,-122.374843,33.675659,-117.868223
4,2018-07-25,Southwest Airlines Co.,DAL,PHL,False,False,1225,1323.0,58.0,58.0,...,46.0,1.0,3.0,1600-1659,6,0.0,32.847114,-96.851772,39.871953,-75.241141


In [60]:
new_df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,origin_latitude,origin_longitude,dest_latitude,dest_longitude
0,2018-07-25,Southwest Airlines Co.,BUF,MCO,False,False,1850,1849.0,0.0,-1.0,...,0.0,0.0,0.0,2100-2159,5,0.0,42.940525,-78.732167,28.428889,-81.316028
1,2018-07-25,Southwest Airlines Co.,BUR,PHX,False,False,1555,1624.0,29.0,29.0,...,42.0,1.0,2.0,1700-1759,2,0.0,34.200619,-118.358497,33.434167,-112.008056
2,2018-07-25,Southwest Airlines Co.,SAT,PHX,False,False,1120,1131.0,11.0,11.0,...,3.0,0.0,0.0,1100-1159,4,0.0,29.533694,-98.469778,33.434167,-112.008056
3,2018-07-25,Horizon Air,SFO,SNA,False,False,926,929.0,3.0,3.0,...,-17.0,0.0,-2.0,1100-1159,2,0.0,37.619002,-122.374843,33.675659,-117.868223
4,2018-07-25,Southwest Airlines Co.,DAL,PHL,False,False,1225,1323.0,58.0,58.0,...,46.0,1.0,3.0,1600-1659,6,0.0,32.847114,-96.851772,39.871953,-75.241141


In [61]:
#Editing values of ArrDelay
new_df['ArrDelayMinuets'] = new_df['ArrDelay'].apply(lambda x: max(0.0,x))
new_df.head()

/var/folders/zv/npgqrjp57ygbmvyh7zhqp9hc0000gn/T/ipykernel_85424/1959714141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ArrDelayMinuets'] = new_df['ArrDelay'].apply(lambda x: max(0.0,x))


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,origin_latitude,origin_longitude,dest_latitude,dest_longitude,ArrDelayMinuets
0,2018-07-25,Southwest Airlines Co.,BUF,MCO,False,False,1850,1849.0,0.0,-1.0,...,0.0,0.0,2100-2159,5,0.0,42.940525,-78.732167,28.428889,-81.316028,0.0
1,2018-07-25,Southwest Airlines Co.,BUR,PHX,False,False,1555,1624.0,29.0,29.0,...,1.0,2.0,1700-1759,2,0.0,34.200619,-118.358497,33.434167,-112.008056,42.0
2,2018-07-25,Southwest Airlines Co.,SAT,PHX,False,False,1120,1131.0,11.0,11.0,...,0.0,0.0,1100-1159,4,0.0,29.533694,-98.469778,33.434167,-112.008056,3.0
3,2018-07-25,Horizon Air,SFO,SNA,False,False,926,929.0,3.0,3.0,...,0.0,-2.0,1100-1159,2,0.0,37.619002,-122.374843,33.675659,-117.868223,0.0
4,2018-07-25,Southwest Airlines Co.,DAL,PHL,False,False,1225,1323.0,58.0,58.0,...,1.0,3.0,1600-1659,6,0.0,32.847114,-96.851772,39.871953,-75.241141,46.0


Now we want to calculate the frequency of flights arriving and leaving from each airport. To do this we have, used SQL to extract the count of destinations that airports are uwing SQL and making in a dataframe and then merging it with the our main datafram df2. 
We did the same thing for arrivals.

In [62]:
#Airport cleaning on the top 10 most busiest airports with origin and destination 

origin_groupby = duckdb.sql('''
SELECT origin, COUNT(origin) AS origin_count
FROM df2
GROUP BY origin
ORDER BY origin_count DESC
''').df()

dest_groupby = duckdb.sql('''
SELECT dest, COUNT(dest) AS dest_count
FROM df2
GROUP BY dest
ORDER BY dest_count DESC
''').df()

In [63]:
#Merge with destination
all_df = duckdb.sql('''
SELECT * 
FROM new_df
JOIN origin_groupby ON new_df.origin=origin_groupby.origin
JOIN dest_groupby ON new_df.dest=dest_groupby.dest
''').df()

all_df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,DivAirportLandings,origin_latitude,origin_longitude,dest_latitude,dest_longitude,ArrDelayMinuets,Origin_2,origin_count,Dest_2,dest_count
0,2018-12-27,Comair Inc.,PHL,TYS,False,False,1754,1831.0,37.0,37.0,...,0.0,39.871953,-75.241141,35.812487,-83.992856,28.0,PHL,8072,TYS,1171
1,2018-12-27,Alaska Airlines Inc.,SEA,HNL,False,False,1748,1741.0,0.0,-7.0,...,0.0,47.448982,-122.309313,21.318691,-157.922407,0.0,SEA,13180,HNL,4923
2,2018-12-27,Capital Cargo International,PHL,CHO,False,False,1449,1444.0,0.0,-5.0,...,0.0,39.871953,-75.241141,38.138639,-78.452861,0.0,PHL,8072,CHO,454
3,2018-12-27,United Air Lines Inc.,SAN,SFO,False,False,1700,1705.0,5.0,5.0,...,0.0,32.733556,-117.189657,37.619002,-122.374843,2.0,SAN,8096,SFO,15839
4,2018-12-27,Alaska Airlines Inc.,PDX,PHX,False,False,830,826.0,0.0,-4.0,...,0.0,45.588722,-122.597500,33.434167,-112.008056,0.0,PDX,6674,PHX,11891


In the cell below we created a column called airport distance which calculates the distance between the airports using the latitude and longitude from the dataset. This will be useful when we do our linear regression in phase 4. 

In [64]:
all_df['airport_distance'] = np.sqrt((all_df['dest_longitude']-all_df['origin_longitude'])**2 
+ (all_df['dest_latitude']-all_df['origin_latitude'])**2)

all_df=all_df.dropna()
all_df = all_df[all_df['airport_distance'] < 250]

print(all_df['airport_distance'])

0          9.647371
1         44.171083
2          3.649592
3          7.124166
4         16.120470
            ...    
568943    12.192258
568944    13.876603
568945    12.192258
568946    12.192258
568947    12.192258
Name: airport_distance, Length: 548760, dtype: float64


# __EXPLORATORY ANALYSIS:__

In the cell below I have calculated the maximum, minimum, standard deviation, mean and median of the times of the delay in departure times in minuets for airplanes in all airports in 2022. The maximum is 7223 minuets which is around 5 Days. Which is a very high ammount. This could possibly be due to a storm or unforceen condition. Possibly in a smaller airports where there are no other flights that can take the passangers, due to which the flight is delayed 5 days. 
The minimum is 0. Suggesting that the flight is not delayed and is on time
The mean is 15.7 minuets. 
The fact that the median for departure is 0 suggests that half of the flights might have not been delayed. 

In [65]:
#Summary Statistics of Delay Analysis
#Departure delay stats;
delay_max = max(all_df['DepDelayMinutes'])
delay_min = min(all_df['DepDelayMinutes'])
delay_std = np.std(all_df['DepDelayMinutes'])
delay_mean = all_df['DepDelayMinutes'].mean()
delay_median = all_df['DepDelayMinutes'].median()
print("This is the max Departure Delay in the year 2022: " + str(delay_max))
print("This is the min Departure Delay in the year 2022: " +str(delay_min))
print("This is the standard deviation Departure Delay in the year 2022: " +str(delay_std))
print("This is the mean Departure Delay in the year 2022: " +str(delay_mean))
print("This is the median Departure Delay in the year 2022: " +str(delay_median))

This is the max Departure Delay in the year 2022: 1723.0
This is the min Departure Delay in the year 2022: 0.0
This is the standard deviation Departure Delay in the year 2022: 43.981057201698064
This is the mean Departure Delay in the year 2022: 13.324533493694876
This is the median Departure Delay in the year 2022: 0.0


In [66]:
#Summary Statistics of Arrival Analysis
#Departure delay stats;
arr_max = max(all_df['ArrDelayMinutes'])
arr_min = min(all_df['ArrDelayMinutes'])
arr_std = np.std(all_df['ArrDelayMinutes'])
arr_mean = all_df['ArrDelayMinutes'].mean()
arr_median = all_df['ArrDelayMinutes'].median()
print("This is the max Arrival Delay in the year 2022: " + str(arr_max))
print("This is the min Arrival Delay in the year 2022: " +str(arr_min))
print("This is the standard deviation Arrival Delay in the year 2022: " +str(arr_std))
print("This is the mean Arrival Delay in the year 2022: " +str(arr_mean))
print("This is the median Arrival Delay in the year 2022: " +str(arr_median))

This is the max Arrival Delay in the year 2022: 1773.0
This is the min Arrival Delay in the year 2022: 0.0
This is the standard deviation Arrival Delay in the year 2022: 43.96775874247014
This is the mean Arrival Delay in the year 2022: 13.62225745316714
This is the median Arrival Delay in the year 2022: 0.0


The plot below describes the Frequency of cancellations per domestic US airline
From the Plot below we can see the the airlines with the most cancellations is Air wisconsin and american airlines, while hawaiian and Horizon have the least number of cancellations. From the plot we can see that biggesr companies like delta, jetblue and american have pretty hih cancellation rates compared to other airlines with american airlines having the most.

In [67]:
#plotting of the the different airlines and frequency of cancellations
## Delay categories per Airline.
filtered_df = all_df[all_df['Cancelled'] == True]
sns.countplot(data=filtered_df, x='Airline', hue='Cancelled')
plt.title('Frequency of Cancellations per Airline')
plt.xlabel('Airline')
plt.ylabel('Frequency')
plt.xticks(rotation=80) 
plt.show()

ValueError: min() arg is an empty sequence

The plot below shows the frequency of Divertions each airline had in 2022. 
From this plot we can see that gojet, american, delta and skywest had the most divertions and the smaller companies like Hawaiian and Horizon had the lest nuymber of divertions.Again from this plot we can see that american airlines had the most number of diversions.
This can help us say that american airlines had the most number of cancellations and divertions in 2022. These could has possibly been due to weather conditions, lack of passengers, lack of staff or any mechanical issues.

In [ ]:
#plotting of the the different airlines and frequency of delays
## Delay categories per Airline.
filtered_df = all_df[all_df['Diverted'] == True]
sns.countplot(data=filtered_df, x='Airline', hue='Diverted')
plt.title('Frequency of Divertions per Airline')
plt.xlabel('Airline')
plt.ylabel('Frequency')
plt.xticks(rotation=80) 
plt.show()

In [ ]:
#DELETE LATER
top_10_origin = origin_groupby.head(10)
top_10_dest = dest_groupby.head(10)

From the above number we can see that the top 10 most popular/ airports with the highest frequency are the same. Now we will plot a graph with this data. This would help us visualize the dat we seen above.

In [ ]:
#top_10_origin
top_10_origin

In [ ]:
#top_10_dest
top_10_dest

In [ ]:
#Now we plot the data from above
joined = top_10_origin.merge(top_10_dest,left_on='Origin',right_on='Dest')
X= joined['Origin']
Y1 = joined['origin_count']
Y2 = joined['dest_count']
  
X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.2, Y1, 0.4, label = 'Origin') 
plt.bar(X_axis + 0.2, Y2, 0.4, label = 'Destination') 
  
plt.xticks(X_axis, X) 
plt.xlabel("Airports") 
plt.ylabel("Delays") 
plt.title("Delays per Airport") 
plt.legend() 
plt.show() 

In [ ]:
#Using the top 10 most busiest airports from above I will see the flight delay. To see if there is any relation
top_10_airports = df2[df2['Origin'].isin(['ATL', 'ORD', 'DFW', 'DEN', 'CTL', 'LAX', 'LGA', 'SEA', 'PHX', 'LAS'])]

In [ ]:
#Time analysis: Depending on the time, how many delays- depending on the month

filtered_df = all_df[all_df['Cancelled'] == True]
sns.countplot(data=filtered_df, x='Month', hue='Cancelled')
plt.title('Frequency of Cancellations per Month')
plt.xlabel('Month')
plt.ylabel('Frequency')
plt.xticks(rotation=80) 
plt.show()

In [ ]:
#Correlation analysis of all our colunms
flights_2018_corr = flights_2018[['Cancelled','Diverted','DepDelayMinutes','DepTime','ArrTime','ArrDelayMinutes','AirTime','Quarter','Month','DayOfWeek','OriginAirportID','DestAirportID']]
matrix = flights_2018_corr.corr()
sns.heatmap(matrix)
print("Correlation matrix is : ")
#print(matrix)

From this correlation matrix, we notice that many of the variables we have selected have low correlation with each other, other than the variables that would make sense to be correlated (if a flight is delayed, then its arrival time will be later). This indicates we should look for other data that more directly correlates to the DepDelayMinutes to be able to eventually make predictions based on the data. 

We originally had planned to use weather data as well, but found that the API we were scraping from only had current data. For future parts of the project, we plan to see if there are other resources we can use that have historical weather data and whether that has a greater correlation.

# __DATA DESCRIPTION:__

__Where can your raw source data be found, if applicable? Provide a link to the raw data.__

Flights Delay Prediction: https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022?select=Combined_Flights_2022.csv

List of US Airports:https://www.kaggle.com/datasets/aravindram11/list-of-us-airports

NWS Weather Data (Using data scraping and API): https://www.weather.gov/documentation/services-web-api




### __MOTIVATION__
__For what purpose was the dataset created?__
- Flight delays are something most of us who have traveled by air have experienced. Regardless of the reason for delays, they pose a huge inconvenience for travelers. These datasets were created so that relevant analysis can be done on the vast amount of data available on flight travel. This would help generate valuable insight into the cause and effect of flight delays. Gaining insight into this would help airline companies, airports, and passengers. 
- The dataset on the list of US airports with geolocation coordinates was created by Aravind Ram Nathan on Kaggle to make plotting on map analysis easier for researchers. 

__Who funded the creation of the dataset?__
- It is unclear who funded the creation of the Flight Status Prediction Dataset, but the data has been extracted from the Marketing Carrier On-Time Performance (Beginning January 2018) data table of the "On-Time" database from the TranStats data library.


### __COMPOSITION__

__Is the data self contained, or does it link to or otherwise rely on external resources ( Like websites, tweets and other databases)__
While, the main dataset we will be working with ( Flight status prediction database) which was extracted from Marketing Carrier On-Time Performance (Beginning January 2018) data table of the "On-Time" database from the TranStats data library, contains most of the data we will be using for our analysis. As mentioned earlier we did also rely on other external sources for data on US Airports and data on weather in different locations in the US from the National Weather Service. 
All the data from the 3 sources are easy to understand and usable form for our analysis.

__What are the observations(rows) and attributes (columns)?__
- *Note all these Column names and descriptions are from the kaggle description of the dataset

__Columns (flight status prediction dataset)__
Year: Year 
Quarter: Quarter (1-4)
Month: Month
DayofMonth: Day of Month
DayOfWeek: Day of Week
FlightDate: Flight Date (yyyymmdd)
Marketing_Airline_Network: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
Operated_or_Branded_Code_Share_Partners: Reporting Carrier Operated or Branded Code Share Partners
DOT_ID_Marketing_Airline: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
IATA_Code_Marketing_Airline: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
Flight_Number_Marketing_Airline: Flight Number
Originally_Scheduled_Code_Share_Airline: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
DOT_ID_Originally_Scheduled_Code_Share_Airline: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
ATA_Code_Originally_Scheduled_Code_Share_Airline: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
Flight_Num_Originally_Scheduled_Code_Share_Airline: Flight Number
Operating_Airline: Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
DOT_ID_Operating_Airline: An identification number assigned by US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
IATA_Code_Operating_Airline: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
Tail_Number: Tail Number
Flight_Number_Operating_Airline: Flight Number
OriginAirportID: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
OriginAirportSeqID: Origin Airport, Airport Sequence ID. An identification number assigned by US DOT to identify a unique airport at a given point of time. Airport attributes, such as airport name or coordinates, may change over time.
OriginCityMarketID: Origin Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.
Origin: Origin Airport
OriginCityName: Origin Airport, City Name
OriginState: Origin Airport, State Code
OriginStateFips: Origin Airport, State Fips
OriginStateName: Origin Airport, State Name
OriginWac: Origin Airport, World Area Code
DestAirportID: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
DestAirportSeqID: Destination Airport, Airport Sequence ID. An identification number assigned by US DOT to identify a unique airport at a given point of time. Airport attributes, such as airport name or coordinates, may change over time.
DestCityMarketID: Destination Airport, City Market ID. City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.
Dest: Destination Airport
DestCityName: Destination Airport, City Name
DestState: Destination Airport, State Code
DestStateFips: Destination Airport, State Fips
DestStateName: Destination Airport, State Name
DestWac: Destination Airport, World Area Code
CRSDepTime: CRS Departure Time (local time: hhmm)
DepTime: Actual Departure Time (local time: hhmm)
DepDelay: Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.
DepDelayMinutes: Difference in minutes between scheduled and actual departure time. Early departures set to 0.
DepDel15: Departure Delay Indicator, 15 Minutes or More (1=Yes)
DepartureDelayGroups: Departure Delay intervals, every (15 minutes from 180)
DepTimeBlk: CRS Departure Time Block, Hourly Intervals
TaxiOut: Taxi Out Time, in Minutes
WheelsOff: Wheels Off Time (local time: hhmm)
WheelsOn: Wheels On Time (local time: hhmm)
TaxiIn: Taxi In Time, in Minutes
CRSArrTime: CRS Arrival Time (local time: hhmm)
ArrTime: Actual Arrival Time (local time: hhmm)
ArrDelay: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
ArrDelayMinutes: Difference in minutes between scheduled and actual arrival time. Early arrivals set to 0.
ArrDel15: Arrival Delay Indicator, 15 Minutes or More (1=Yes)
ArrivalDelayGroups: Arrival Delay intervals, every (15-minutes from 180)
ArrTimeBlk: CRS Arrival Time Block, Hourly Intervals
Cancelled: Cancelled Flight Indicator (1=Yes)
CancellationCode: Specifies The Reason For Cancellation
Diverted: Diverted Flight Indicator (1=Yes)
CRSElapsedTime: CRS Elapsed Time of Flight, in Minutes
ActualElapsedTime: Elapsed Time of Flight, in Minutes
AirTime: Flight Time, in Minutes
Flights: Number of Flights
Distance: Distance between airports (miles)
DistanceGroup: Distance Intervals, every 250 Miles, for Flight Segment
CarrierDelay: Carrier Delay, in Minutes
WeatherDelay: Weather Delay, in Minutes
NASDelay: National Air System Delay, in Minutes
SecurityDelay: Security Delay, in Minutes
LateAircraftDelay: Late Aircraft Delay, in Minutes
FirstDepTime: First Gate Departure Time at Origin Airport
TotalAddGTime: Total Ground Time Away from Gate for Gate Return or Cancelled Flight
LongestAddGTime: Longest Time Away from Gate for Gate Return or Cancelled Flight
DivAirportLandings: Number of Diverted Airport Landings
DivReachedDest: Diverted Flight Reaching Scheduled Destination Indicator (1=Yes)
DivActualElapsedTime: Elapsed Time of Diverted Flight Reaching Scheduled Destination, in Minutes. The ActualElapsedTime column remains NULL for all diverted flights.
DivArrDelay: Difference in minutes between scheduled and actual arrival time for a diverted flight reaching scheduled destination. The ArrDelay column remains NULL for all diverted flights.
DivDistance: Distance between scheduled destination and final diverted airport (miles). Value will be 0 for diverted flight reaching scheduled destination.
Div1Airport: Diverted Airport Code1
Div1AirportID: Airport ID of Diverted Airport 1. Airport ID is a Unique Key for an Airport
Div1AirportSeqID: Airport Sequence ID of Diverted Airport 1. Unique Key for Time Specific Information for an Airport
Div1WheelsOn: Wheels On Time (local time: hhmm) at Diverted Airport Code1
Div1TotalGTime: Total Ground Time Away from Gate at Diverted Airport Code1
Div1LongestGTime: Longest Ground Time Away from Gate at Diverted Airport Code1
Div1WheelsOff: Wheels Off Time (local time: hhmm) at Diverted Airport Code1
Div1TailNum: Aircraft Tail Number for Diverted Airport Code1
Div2Airport: Diverted Airport Code2
Div2AirportID: Airport ID of Diverted Airport 2. Airport ID is a Unique Key for an Airport
Div2AirportSeqID: Airport Sequence ID of Diverted Airport 2. Unique Key for Time Specific Information for an Airport
Div2WheelsOn: Wheels On Time (local time: hhmm) at Diverted Airport Code2
Div2TotalGTime: Total Ground Time Away from Gate at Diverted Airport Code2
Div2LongestGTime: Longest Ground Time Away from Gate at Diverted Airport Code2
Div2WheelsOff: Wheels Off Time (local time: hhmm) at Diverted Airport Code2
Div2TailNum: Aircraft Tail Number for Diverted Airport Code2
Div3Airport: Diverted Airport Code3
Div3AirportID: Airport ID of Diverted Airport 3. Airport ID is a Unique Key for an Airport
Div3AirportSeqID: Airport Sequence ID of Diverted Airport 3. Unique Key for Time Specific Information for an Airport
Div3WheelsOn: Wheels On Time (local time: hhmm) at Diverted Airport Code3
Div3TotalGTime: Total Ground Time Away from Gate at Diverted Airport Code3
Div3LongestGTime: Longest Ground Time Away from Gate at Diverted Airport Code3
Div3WheelsOff: Wheels Off Time (local time: hhmm) at Diverted Airport Code3
Div3TailNum: Aircraft Tail Number for Diverted Airport Code3
Div4Airport: Diverted Airport Code4
Div4AirportID: Airport ID of Diverted Airport 4. Airport ID is a Unique Key for an Airport
Div4AirportSeqID: Airport Sequence ID of Diverted Airport 4. Unique Key for Time Specific Information for an Airport
Div4WheelsOn: Wheels On Time (local time: hhmm) at Diverted Airport Code4
Div4TotalGTime: Total Ground Time Away from Gate at Diverted Airport Code4
Div4LongestGTime: Longest Ground Time Away from Gate at Diverted Airport Code4
Div4WheelsOff: Wheels Off Time (local time: hhmm) at Diverted Airport Code4
Div4TailNum: Aircraft Tail Number for Diverted Airport Code4
Div5Airport: Diverted Airport Code5
Div5AirportID: Airport ID of Diverted Airport 5. Airport ID is a Unique Key for an Airport
Div5AirportSeqID: Airport Sequence ID of Diverted Airport 5. Unique Key for Time Specific Information for an Airport
Div5WheelsOn: Wheels On Time (local time: hhmm) at Diverted Airport Code5
Div5TotalGTime: Total Ground Time Away from Gate at Diverted Airport Code5
Div5LongestGTime: Longest Ground Time Away from Gate at Diverted Airport Code5
Div5WheelsOff: Wheels Off Time (local time: hhmm) at Diverted Airport Code5
Div5TailNum: Aircraft Tail Number for Diverted Airport Code5
Duplicate: Duplicate flag marked Y if the flight is swapped based on Form-3A data

__Rows (flight status prediction dataset)__
Each row represents the information that is described in each column for a distinct flight

__Colunms (List of US airports)__
IATA: 3 letter acronym of the US airport 
AIRPORT: Name of the airport
CITY: Name of the US City the airport is located in 
STATE: Name of the US state the airport is located in
COUNTRY: Country the airport is located in 
LATITUDE: Latitude coordinated of the airport 
LONGITUDE: Longitude coordinated of the airport

__Rows (List of US airports)__
Each row represents the information that is described in each column for a distinct Airport in the US
Colunms (Weather Data)
Rows (Weather Data)




### __COLLECTION PROCESS__
__What processes might have influenced what data was observed and recorded and what was not?__
The data that is in this dataset is obtained from the TranStats data library, Public kaggle and National Weather Service. These are all  govermental organizations and public websites, that collect data for public use. In terms of what might have influenced what data was observed and recorded, we can say that this is readily available non personal, not private data that would not breach and acts of privacy. As opposed to any personal data, that contains personal information like phone number, age, name, salary or home address. 
Moreover, some of the airports and airlines operate international flights, this type of data would be ignored or any private jets which go to private airports would be ignored. Even flights that don’t carry passsengers and only carry goods would be ignored for this dataset. 

__Over what timeframe was the data collected?__
The flight delay prediction data was collected from 2018 - 2022. However for this analysis we only used data from 2022. 
The List of US airports data has no time attribute to it, but was last updated on kaggle 3 years ago from 2023. 


__What mechanisms or procedures were used to collect the data?__
The flights delay prediction and list of US airports was obtained from online sources which was then collected and uploaded by the creator on kaggle

__What preprocessing was done, and how did the data come to be in the form that you are using?__
We first concatenated the two datasets with the first half of 2022 and the second half of 2022, which we named flights_2022. Because these are domestic flights, we dropped the 'country' column. Then, we joined the dataset of airports to flights_2022 using duckdb SQL, while selecting only the columns of these tables that were relevant. Then, while scanning through the arrival minute times, we realized some values were negative, so we converted those to 0. 

We originally had scraped weather data as well, which is included in a separate file, but we are no longer using this. We may scrape weather data from a different API in the future and join it to flights_2022. 

__Has the dataset been used for any tasks already?__
The main dataset is a dataset from Kaggle so people have used it for competitions and exploration. However, we do not have access to their work.

There has also been a lot of analysis done on this topic, but from a different dataset at different times.

__If people are involved, were they aware of the data collection and if so,what purpose did they expect the data to be used for?__
They are aware because they uploaded the data to Kaggle for public use.

# PHASE 4

**Linear Regression** <br>

**Hypothesis 1:** Flights with a larger distance, calculated from the longitudes and latitudes 
of the origins and destinations, are more likely to be delayed than flights with a smaller distance.


We first split the data into train and test set after which we created a linear regression, where the input is the airport distance calculated in the last part of data cleaning. We used that regression and the X and y test sets to make predictions (of the delay time) for the remaining X test values (distances).

In [ ]:
train, test = train_test_split(all_df, test_size = 0.20, random_state = 2950)

In [ ]:
X_train = train['airport_distance'].values.reshape(-1,1)
y_train = train['DepDelayMinutes']

model = LinearRegression().fit(X_train, y_train)
model.score(X_train,y_train)

X_test = test['airport_distance'].values.reshape(-1, 1)
y_test = test['DepDelayMinutes']

predictions = model.predict(X_test)

We did some additional exploratory analysis to compare the distance to the delay time, and plotted the linear regression line to visualize the general trend. We noticed that although it appeared to have a negative correlation from the data points, the linear regression line was nearly horizontal, because so many data points were concentrated towards lower delay times which skewed the predictions to be within a very small range.

In [ ]:
plt.scatter(X_train, y_train)
plt.plot(X_test, predictions,color='k')
plt.xlabel('Distance')
plt.ylabel('Delay Time (min)')

We also plotted the actual and predicted values to see how they compared, and realized that the predicted values were within a range of about 1.5, even though many of the actual values spanned between 0 and 600. It is apparent that our model is not going to be a very accurate predictor.

In [ ]:
plt.scatter(y_test, predictions)
plt.title('Actual vs. Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

We also created a residuals plot, which had very high residuals, so we know that the model is not representative of the data. 

In [ ]:
residuals = y_test - predictions
plt.scatter(predictions, residuals, marker='x', color='red', alpha=0.7)

We can conclude that our hypothesis is incorrect because through our prediction analysis, we did not find any correlation between airport distance and delay time. 

**Logistic Regression** <br>

__Hypothesis 2:__ Flights scheduled during the winter months(December, January, February) are more likely to be delayed compared to flights  scheduled during other months of the year. 

We made use of the delayed flights column and not the cancelled as while doing the logistic regression we realized that all the value for if a flight had been cancled is false. Meaning that when we converted it to a boolen it became zero. This could not work for our logistic regression, as we need both 0 and 1

We have made use of month data from the date as we were not able to get weather data due to the API. That is why we decided to use month data from the date as we think this is the closest we can get to using weather as a factor for our analysis. 

In the cell below we are making use of pd.to_datetime to convert the date values in the FlightsDate column in all_df into the same datatype of datetime64.

In [ ]:
all_df["FlightDate"] = pd.to_datetime(all_df["FlightDate"])
print(all_df["FlightDate"].head())

In [ ]:
print(len(all_df))
print(len(pd.DatetimeIndex(all_df['FlightDate']).month))

In the cell below we are making use of pd.DatetimeIndex to extract the month from the flightdate

In [ ]:
all_df['flight_month'] = pd.DatetimeIndex(all_df['FlightDate']).month
print(all_df['flight_month'].head())

Now we are mapping each of the numerical months. Example where 1 means January. We are mapping the numerical representations of the months to their engligh months. This make the analysis and understanding much easier.

In [ ]:
month_mapping = {
    1: 'Jan',2: 'Feb',3: 'March', 
    4: 'April', 5: 'May', 6:'June', 7: 'July', 
    8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}

all_df['flight_month'] = all_df['flight_month'].map(month_mapping)

print(all_df['flight_month'].head())

In [ ]:
all_df['Cancelled_bool'] = all_df['Diverted'].astype(int)

In the cell above we just convert the values that are a boolen of True or False into 0's and 1's. 

In the cell below we use flight month as an input to our classifier, we need to convert the flight month variable into 12 different dummy variables. For example, one of the new dummy months march, and will equal 1 if that row's month is march, and 0 if that row's month value is not march.
To do this we use pd.get_dummies().

In [ ]:
month_dummies = pd.get_dummies(all_df['flight_month'], 
                               dtype=int, drop_first = False) 
print(month_dummies.head())

In [ ]:
month_with_dummies = pd.concat([all_df, month_dummies ],axis=1)
print(month_with_dummies.head(5))

In [ ]:
Now we will split our dataset into train and test. This will help us while doing our linear regression. 

In [ ]:

from sklearn.model_selection import train_test_split
flight_cancelled_train, flight_cancelled_test = train_test_split(month_with_dummies , 
                                             test_size=0.20 , random_state=2950)

Now we want to train a logistic regression model to predict whether a movie is cancelled. We use a logistic regression model that predicts cancelled_bool given the month ( Jan, Feb, March, April, May, June, July, Aug, Sept, Oct, Nov, Dec).
I have defined the Normalizer function from HW4

In [ ]:
#code
def Normalizer(df_cols):
    scaler = preprocessing.StandardScaler().fit(df_cols)
    return(scaler.transform(df_cols))
from sklearn.linear_model import LogisticRegression
input_columns = ['Jan', 'Feb', 'March', 
                 'April', 'May', 'June', 'July', 'Aug','Sept','Oct','Nov','Dec']
X = flight_cancelled_train[input_columns]
y = flight_cancelled_train['Cancelled_bool']
logistic_model = LogisticRegression(random_state=16).fit(Normalizer(X),y)

Now to analyze our model, data and see if our hypothesis is correct we will print the coeff for all the months. We will do this in a table. This make it easier for us to read and understand the values. From the coeff we can figure out which has the highest correlation with the delays.

To do this we first Create lists to store month names and coefficients. Then we Iterate through coefficients and append them to the list. after which we Create a DataFrame with months and coefficients. Below is the printed resulting dataframe. 

In [ ]:

coefficients = logistic_model.coef_[0]


months = ['Jan', 'Feb', 'March', 'April', 
          'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
coefficient_values = []


for x in range(len(coefficients)):
    coefficient_values.append(round(logistic_model.coef_[0][x],3))


result_df = pd.DataFrame({
    'Month': months,
    'Coefficient': coefficient_values
})


print(result_df)

From the above table I will interpret the results then link it to my hypothesis. 


1. Our model estimates that, all else equal, if a flight is flying during the month `January`, the odds of the fligh being delayed are decreased, the odds are multiplied by around -0.050 $(e^{-0.050})$
2. Our model estimates that, all else equal, if a flight is flying during the month `Febuary`, the odds of the fligh being delayed are increased, the odds are multiplied by around 0.029 $(e^{0.029})$
3. Our model estimates that, all else equal, if a flight is flying during the month `December`, the odds of the fligh being delayed are increased, the odds are multiplied by around 0.007 $(e^{0.007})$
4. Our model estimates that, all else equal, if a flight is flying during the month `August`, the odds of the fligh being delayed are increased, the odds are multiplied by around 0.140 $(e^{0.140})$
5. Our model estimates that, all else equal, if a flight is flying during the month `May`, the odds of the fligh being delayed are increased, the odds are multiplied by around 0.064 $(e^{0.064})$

**While our initial hypothesis posited that flights scheduled during the winter months would be more prone to delays, the results of our logistic regression model have introduced interesting nuances. Contrary to expectations, certain months outside the winter season (May, Au and July) exhibit a higher likelihood of delays, challenging our initial assumptions.**

Now we will use the test set to predict our values of y predicted

In [ ]:
#Evaluate the test data and predict 
y_pred = logistic_model.predict(Normalizer(flight_cancelled_test[input_columns]))
# Print the first ten values of y_pred
print(y_pred[:10])

In [ ]:
y_true = flight_cancelled_test['Cancelled_bool']
# Print accuracy
print(f"accuracy: {accuracy_score(y_true, y_pred)}")
# Print precision
print(f"precision: {precision_score(y_true, y_pred):.2f}")
# Print recall
print(f"recall: {recall_score(y_true, y_pred):.2f}")
# Print F1
print(f"F1: {f1_score(y_true, y_pred):.2f}")

**Interpreting our results form above from the accuracy, precision, recall score and f1 score**

1. Accuracy: 99.7% of the test data are predicted correctly (i.e. 99.7% of the test data are either true positives, where a true positive is delayed (1) and is predicted to be delayed, or true negatives, where a flight is not delayed (0) and is predicted as not delayed.) From the results above I would say the analysis for 99.7% accuracy is mostly because of a flight that not delayed and is predicted as not delayed due to the high number of 0. 
2. A precision of 0.00 means that the model did not correctly identify any instances of the positive class. This can happen when the model classifies everything as the majority class (e.g., class 0).
3. A recall of 0.00 indicates that the model missed all instances of the positive class.
4. Like precision and recall, an F1 score of 0.00 suggests that the model is not effectively identifying the positive class.



The main issue I have identified for this could be because our model is classifying most instances as the majority class (class 0) and not successfully capturing the positive instances. This could be because our traning dataset is flawed. Most of our data for the flight being delayed is 0. This creates a bias as our model is trained with the majority class zero. This is why we are having issues.

**The main issue is class imbalance, where the majority class (class 0, likely representing non-delayed flights) dominates the training dataset. This imbalance can lead the model to be biased towards predicting the majority class, resulting in high accuracy but poor performance on the positive class (delayed flights).**

**Multiple Linear Regression**
__Hypothesis 3:__ Flights scheduled during the extreme weather conditions are more likely to be delayed compared to flights scheduled during normal conditions. 

In order to proceed with this analysis and get weather data. From out data analysis above we toook the airports: ORD, DEN, ATL and LAX. From our EDA we can see that these for busiest airports with the most delays in the US. 
Using these four airports we scraped weather data (temperature, dew point, humidity, wind, Wind speed, Winds Gust, Pressure, precipitation and condition). 
Below is an example of the URL that we scarped for Denvers data: 

https://www.wunderground.com/history/daily/KDEN/date (https://www.wunderground.com/history/daily/KDEN/date)

After we scraped the data, we saved this in a file called weather data in our GITHUB. 
We will be using these values for our inputs for the multiple Linear regression. After we make the model for our multiple linear regression. 

We can take a a look at the appendix for the web scraping data.

# PHASE 5

In [203]:
weather_atl_df = pd.read_csv("weather-data/ATL/atl_hourly.csv")
weather_ord_df = pd.read_csv("weather-data/ORD/ord_hourly.csv")
weather_den_df = pd.read_csv("weather-data/DEN/den_hourly.csv")
weather_lax_df = pd.read_csv("weather-data/LAX/lax_hourly.csv")


In [204]:

combined_df = duckdb.sql('''
           
SELECT 
  Date,
  AVG(CAST(Temperature AS DOUBLE)) as avg_temp,
  AVG(CAST(Dew_Point AS DOUBLE)) as avg_dew_point,
  AVG(CAST(Humidity AS DOUBLE)) as avg_humidity,
  AVG(CAST(Wind_Speed AS DOUBLE)) as avg_wind_speed,
  AVG(CAST(Wind_Gust AS DOUBLE)) as avg_wind_gust,
  AVG(CAST(Pressure AS DOUBLE)) as avg_pressure,
  AVG(CAST(Precip AS DOUBLE)) as avg_precip,
  all_df.DepDelayMinutes as dep_delay_minutes
FROM weather_df
LEFT JOIN all_df ON weather_df.Date=all_df.FlightDate
WHERE all_df.ORIGIN = 'ATL'
GROUP BY Date, DepDelayMinutes
                    
    
          ''').df()

In [205]:
combined_df.head()

,Date,avg_temp,avg_dew_point,avg_humidity,avg_wind_speed,avg_wind_gust,avg_pressure,avg_precip,dep_delay_minutes
0,2018-05-12,57.750000,50.044118,50.044118,10.029412,0.404412,27.829044,0.007353,0.0
1,2018-05-13,59.212766,50.971631,50.971631,8.042553,0.000000,27.785816,0.011348,35.0
2,2018-05-13,59.212766,50.971631,50.971631,8.042553,0.000000,27.785816,0.011348,25.0
3,2018-05-13,59.212766,50.971631,50.971631,8.042553,0.000000,27.785816,0.011348,27.0
4,2018-05-13,59.212766,50.971631,50.971631,8.042553,0.000000,27.785816,0.011348,1.0


In [206]:
combined_train, combined_test = train_test_split(combined_df, test_size=0.1, random_state=2950)

In [209]:
input_vars = ["avg_temp", "avg_dew_point", "avg_humidity","avg_wind_speed","avg_wind_gust","avg_pressure","avg_precip"]

X = combined_train[input_vars]
y = combined_train['dep_delay_minutes']

model=LinearRegression().fit(X,y)

for var_name, var_coef in zip(input_vars, model.coef_):
    print(f"{var_name}: {round(var_coef,2)}")

print(f"intercept: {round(model.intercept_,2)}")

avg_temp: -0.87
avg_dew_point: -2562080322398.96
avg_humidity: 2562080322399.82
avg_wind_speed: -0.1
avg_wind_gust: 1.38
avg_pressure: 19.65
avg_precip: 293.64
intercept: -511.94


In [215]:
train_predictions = model.predict(combined_train[input_vars])
test_predictions = model.predict(combined_test[input_vars])

In [217]:
train_rmse = np.sqrt(np.mean((train_predictions - combined_train["dep_delay_minutes"])**2))
test_rmse = np.sqrt(np.mean((test_predictions - combined_test["dep_delay_minutes"])**2))

# Print train RMSE
print(f"Train RMSE: {round(train_rmse, 2)}")
# Print test RMSE
print(f"Test RMSE: {round(test_rmse, 2)}")

Train RMSE: 61.11
Test RMSE: 61.01


In [219]:
train_mae = np.mean(np.abs(train_predictions - combined_train["dep_delay_minutes"]))
test_mae = np.mean(np.abs(test_predictions - combined_test["dep_delay_minutes"]))

# Print train MAE
print(f"Train MAE: {round(train_mae, 2)}")
# Print test MAE
print(f"Test MAE: {round(test_mae, 2)}")

Train MAE: 33.11
Test MAE: 32.19


# CONCLUSION AND INTERPRETATIONS

Our project explored what factors best predict flight delays and cancellations in the United States. We used flight data from 2018 because it was a complete dataset from a year that did not overlap with COVID which would produce different trends in delays and cancellations. Our dataset included nearly 5 million flights which was too large to work with, so we took a 10% random sample of the data. Each row represented a flight, and included columns that defined the origin and destination airports and their locations, the length of the delay of the flight, whether the flight was cancelled or not cancelled, and other variables which could be used as predictors. 

Our first hypothesis was that flights with a larger distance, calculated from the longitudes and latitudes of the origins and destinations, are more likely to be delayed than flights with a smaller distance. We conclude that flights with a larger distance are not more likely to be delayed than flights with a smaller distance. There is no direct correlation between flight distance and delay time of flights. Our data showed that flight delay times are very concentrated around very slight delays, so it is difficult to make predictions about flight delay time from just the distance between airports because predicted values were concentrated around the same delay times. 

Our second hypothesis was that flights scheduled during the winter months(December, January, February) are more likely to be delayed compared to flights scheduled during other months of the year. While our initial hypothesis posited that flights scheduled during the winter months would be more prone to delays, the results of our logistic regression model have introduced interesting nuances. Contrary to expectations, certain months outside the winter season (May, Aug and July) exhibit a higher likelihood of delays, challenging our initial assumptions.

# __DATA LIMITATIONS__:

__The main limitations of our data set that have been noticed are:__
1) The dataset also does not have many descriptive variables that affect the delays or cancellations. This means that we are not able to find high correlation values and therefore will struggle to create a predictive model or generate any meaningful insights. 
2) Because of US weather api issues we were not able to get any data on US weather. This created issues as we were not able to improve our analysis and add more depth to our analysis on weather data. This affected our analysis as we were not able to make a more comprehensive analysis and predict to see if flight delays and cancellation is affe cted by weather. 
3) Another issue we faced was that our flight cancellation and delay column most of the entries are false. When we convert it to a binary, most of the values will be zero. This creates an issue when we trained our logistic regression. Creating class imbalance, where the majority class (class 0, likely representing non-delayed flights) dominates the training dataset. This imbalance can lead the model to be biased towards predicting the majority class, resulting in high accuracy but poor performance on the positive class (delayed flights). 

# FUTURE RESEARCH

Future research could include using a multiple regression with more variables in addition to distance to see if we can get a more accurate prediction of the delay times. Additionally, this could include weather data if a future research study had funding to purchase data from the weather API. We could also use international data rather than just data of flights in the US. 

# __Refrences:__

1. https://medium.com/analytics-vidhya/modeling-flight-delays-through-u-s-flight-data-2f0b3d7e2c89
2. https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022?select=Combined_Flights_2022.csv 
3. https://www.kaggle.com/datasets/aravindram11/list-of-us-airports 


# __Appendix:__


Below is the code used to sample 10% of our large datset to get data for flights in 2018. 

In [ ]:
#df1 = pd.read_csv('Combined_Flights_2018.csv')
#df2 = pd.read_csv('Combined_Flights_2019.csv')
#df1.head()

In [ ]:
#print(len(df1.index))

In [ ]:
#smaller_df = pd.DataFrame()

#for month in range(1,13):
    #sample = df1[df1['Month'] == month].sample(frac=.1)
    #smaller_df = pd.concat([smaller_df, sample])

#smaller_df.head()
#print(len(smaller_df.index))

In [ ]:
#smaller_df = smaller_df.sort_values('FlightDate')

In [ ]:
#smaller_df.to_csv(r'/Users/joshuapark/Downloads/test-2950/Flights_2018_Modified.csv')

Below is the code used to scrape weather data from Weather Underground

In [ ]:
# import csv
# import time
# import datetime
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import pandas as pd


# date = datetime.datetime(2018,12,3)

# endDate = datetime.datetime(2018,12,3)

# prefixURL = 'https://www.wunderground.com/history/daily/KLAX/date/'
# # prefixURL = 'https://www.wunderground.com/history/daily/KATL/date/'
# # prefixURL = 'https://www.wunderground.com/history/daily/KDEN/date/'
# # prefixURL = 'https://www.wunderground.com/history/daily/KORD/date/'


# def scrape_page(date, prefixURL, sleepTime, errorOccurs):
#         print("")
#         print("Function started:", time.strftime("%H:%M:%S", time.localtime()))     
#         url = ''.join([prefixURL+str(date.date())])
#         print("scraping:", url)

#         driver = webdriver.Chrome()
#         driver.get(url)
#         tables = WebDriverWait(driver,2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))

#         time.sleep(sleepTime) #obey robots.txt, adds time to reduce errors

#         a = pd.read_html(tables[1].get_attribute('outerHTML'))
#         df=a[0]
#         df = df[df['Time'].notna()] #drops rows
#         df.insert(0, 'Date', date, allow_duplicates=True)
#         print(df)
#         df.to_csv(r'temps.csv', index = False, columns=['Date','Time','Temperature','Dew Point','Humidity','Wind','Wind Speed','Wind Gust','Pressure','Precip.','Condition'], mode='a', header=None)
#         print("There are", len(df), "rows for", str(date.date()))
#         with open('NumEntries.csv', 'a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([str(date.date()),len(df)])


# errorTracker= 0
# MAXERROR = 1

# while date <= endDate:
#     if errorTracker >=MAXERROR:
#         print("There were ", errorTracker, " errors in a row, so the program stopped. Check the NumEntriesFile")
#         # break
#         date += datetime.timedelta(days=1)

#     try:
#         scrape_page(date,prefixURL,sleepTime=3, errorOccurs=False)
#         date += datetime.timedelta(days=1)   #increments date by one
#         errorTracker=0 # returns errorTracker to 0

#     #### If the scrape_page() function runs and fails then the date is not incremented

#     except IndexError as e:
#         print("AN ERROR OCCURRED", str(e)) 
#         time.sleep(5)  #wait 
#         with open('NumEntries.csv', 'a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(["NEW ERROR, did not get df",str(e),str(date.date())])
#         errorTracker += 1

#     except:   # spotty wifi
#         print("AN ERROR OCCURRED") 
#         time.sleep(5)  #wait if timoutexception raised and try again, spotty wifi
#         with open('NumEntries.csv', 'a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(["NEW ERROR, did not get df",str(date.date())])
#         errorTracker += 1

